# TPCH - 1

In [1]:
from datetime import datetime, timedelta
import pysdql

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)

d1 = datetime.strptime('1998-12-01', "%Y-%m-%d")
d3 = d1 + timedelta(days=76)
var1 = d3.strftime("%Y-%m-%d")

r = lineitem[(lineitem['l_shipdate'] <= var1)]
r = r.groupby(['l_returnflag', 'l_linestatus'])
r = r.aggr(sum_qty=(r['l_quantity'], 'sum'),
           sum_base_price=(r['l_extendedprice'], 'sum'),
           sum_disc_price=(r['l_extendedprice'] * (1 - r['l_discount']), 'sum'),
           sum_charge=(r['l_extendedprice'] * (1 - r['l_discount']) * (1 + r['l_tax']), 'sum'),
           avg_qty=(r['l_quantity'], 'avg'),
           avg_price=(r['l_extendedprice'], 'avg'),
           avg_disc=(r['l_discount'], 'avg'),
           count_order=('*', 'count'))
db_driver.run(r)

let lineitem = { < l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 1, l_quantity = 17, l_extendedprice = 15317.0, l_discount = 0.04, l_tax = 0.02, l_returnflag = "N", l_linestatus = "O", l_shipdate = 19960313, l_commitdate = 19960212, l_receiptdate = 19960322, l_shipinstruct = "DELIVER IN PERSON", l_shipmode = "TRUCK", l_comment = "egular courts above the" > -> 1, 
< l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 2, l_quantity = 36, l_extendedprice = 32436.0, l_discount = 0.09, l_tax = 0.06, l_returnflag = "N", l_linestatus = "O", l_shipdate = 19960412, l_commitdate = 19960228, l_receiptdate = 19960420, l_shipinstruct = "TAKE BACK RETURN", l_shipmode = "MAIL", l_comment = "ly final dependencies: slyly bold " > -> 1, 
< l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 3, l_quantity = 8, l_extendedprice = 7208.0, l_discount = 0.1, l_tax = 0.02, l_returnflag = "R", l_linestatus = "O", l_shipdate = 19960129, l_commitdate = 19960305, l_receiptdate = 19

{<l_returnflag = R, l_linestatus = O, sum_qty = 8, sum_base_price = 7208.0, sum_disc_price = 6487.2, sum_charge = 6616.9439999999995, avg_qty = 8.0, avg_price = 7208.0, avg_disc = 0.1, count_order = 1> -> 1, <l_returnflag = N, l_linestatus = O, sum_qty = 73, sum_base_price = 65773.0, sum_disc_price = 60979.68, sum_charge = 62588.5056, avg_qty = 24.333333333333332, avg_price = 21924.333333333332, avg_disc = 0.07, count_order = 3> -> 1}

# TPCH - 3

In [2]:
import pysdql

var1 = 'BUILDING'
var2 = '1995-03-22'

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

customer = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/customer.tbl', header=pysdql.CUSTOMER_COLS)
orders = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/orders.tbl', header=pysdql.ORDERS_COLS)
lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)

r = customer.merge(orders, on=(customer['c_custkey'] == orders['o_custkey']))
r = r.merge(lineitem, on=(r['o_orderkey'] == lineitem['l_orderkey']))

r = r[(r['c_mktsegment'] == var1)
      & (r['o_orderdate'] < var2)
      & (r['l_shipdate'] > var2)]

r = r.groupby(['l_orderkey', 'o_orderdate', 'o_shippriority']) \
    .aggr(revenue=((r['l_extendedprice'] * (1 - r['l_discount'])), 'sum'))

db_driver.run(r)

let customer = { < c_custkey = 1, c_name = "Customer#000000001", c_address = "IVhzIApeRb ot,c,E", c_nationkey = 15, c_phone = "25-989-741-2988", c_acctbal = 711.56, c_mktsegment = "BUILDING", c_comment = "to the even, regular platelets. regular, ironic epitaphs nag e" > -> 1 }
let orders = { < o_orderkey = 1, o_custkey = 1, o_orderstatus = "O", o_totalprice = 124828.11, o_orderdate = 19960102, o_orderpriority = "5-LOW", o_clerk = "Clerk#000000951", o_shippriority = 0, o_comment = "nstructions sleep furiously among " > -> 1 }
let lineitem = { < l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 1, l_quantity = 17, l_extendedprice = 15317.0, l_discount = 0.04, l_tax = 0.02, l_returnflag = "N", l_linestatus = "O", l_shipdate = 19960313, l_commitdate = 19960212, l_receiptdate = 19960322, l_shipinstruct = "DELIVER IN PERSON", l_shipmode = "TRUCK", l_comment = "egular courts above the" > -> 1, 
< l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 2, l_quantity = 36, l_ext

{<l_orderkey = 1, o_orderdate = 19960102, o_shippriority = 0, revenue = 89232.12580000001> -> 1}

# TPCH - 4

In [3]:
import pysdql

var1 = '1996-05-01'
var2 = '1996-08-01'  # var1 + 3 month

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)
orders = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/orders.tbl', header=pysdql.ORDERS_COLS)

r = lineitem[(lineitem['l_commitdate'] < lineitem['l_receiptdate'])]
r = r.merge(orders, on=(r['l_orderkey'] == orders['o_orderkey']))

s = orders[(orders['o_orderdate'] >= var1) & (orders['o_orderdate'] < var2) & r.exists()]

s = s.groupby(['o_orderpriority']).aggr(order_count=('*', 'count'))

db_driver.run(s)

let lineitem = { < l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 1, l_quantity = 17, l_extendedprice = 15317.0, l_discount = 0.04, l_tax = 0.02, l_returnflag = "N", l_linestatus = "O", l_shipdate = 19960313, l_commitdate = 19960212, l_receiptdate = 19960322, l_shipinstruct = "DELIVER IN PERSON", l_shipmode = "TRUCK", l_comment = "egular courts above the" > -> 1, 
< l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 2, l_quantity = 36, l_extendedprice = 32436.0, l_discount = 0.09, l_tax = 0.06, l_returnflag = "N", l_linestatus = "O", l_shipdate = 19960412, l_commitdate = 19960228, l_receiptdate = 19960420, l_shipinstruct = "TAKE BACK RETURN", l_shipmode = "MAIL", l_comment = "ly final dependencies: slyly bold " > -> 1, 
< l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 3, l_quantity = 8, l_extendedprice = 7208.0, l_discount = 0.1, l_tax = 0.02, l_returnflag = "R", l_linestatus = "O", l_shipdate = 19960129, l_commitdate = 19960305, l_receiptdate = 19

{<o_orderpriority = 5-LOW, order_count = 1> -> 1}

# TPCH - 5

In [4]:
import pysdql

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

var1 = 'MIDDLE EAST'
var2 = '1993-01-01'
var3 = '1994-01-01'  # var2 + 1 year

customer = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/customer.tbl', header=pysdql.CUSTOMER_COLS)
orders = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/orders.tbl', header=pysdql.ORDERS_COLS)
lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)
supplier = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/supplier.tbl', header=pysdql.SUPPLIER_COLS)
nation = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/nation.tbl', header=pysdql.NATION_COLS)
region = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/region.tbl', header=pysdql.REGION_COLS)

r = customer.merge(right=orders, on=(customer['c_custkey'] == orders['o_custkey']))
r = r.merge(lineitem, on=(r['o_orderkey'] == lineitem['l_orderkey']))
r = r.merge(supplier, on=((r['l_suppkey'] == supplier['s_suppkey'])
                          & (r['c_nationkey'] == supplier['s_nationkey'])
                          )
            )
r = r.merge(nation, on=(r['s_nationkey'] == nation['n_nationkey']))
r = r.merge(region, on=(r['n_regionkey'] == region['r_regionkey']))

r = r[(region['r_name'] == var1)
      & (orders['o_orderdate'] >= var2)
      & (orders['o_orderdate'] < var3)]

r = r.groupby(['n_name']).aggr(revenue=((lineitem['l_extendedprice'] * (1 - lineitem['l_discount'])), 'sum'))

db_driver.run(r)

let customer = { < c_custkey = 1, c_name = "Customer#000000001", c_address = "IVhzIApeRb ot,c,E", c_nationkey = 15, c_phone = "25-989-741-2988", c_acctbal = 711.56, c_mktsegment = "BUILDING", c_comment = "to the even, regular platelets. regular, ironic epitaphs nag e" > -> 1 }
let orders = { < o_orderkey = 1, o_custkey = 1, o_orderstatus = "O", o_totalprice = 124828.11, o_orderdate = 19960102, o_orderpriority = "5-LOW", o_clerk = "Clerk#000000951", o_shippriority = 0, o_comment = "nstructions sleep furiously among " > -> 1 }
let lineitem = { < l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 1, l_quantity = 17, l_extendedprice = 15317.0, l_discount = 0.04, l_tax = 0.02, l_returnflag = "N", l_linestatus = "O", l_shipdate = 19960313, l_commitdate = 19960212, l_receiptdate = 19960322, l_shipinstruct = "DELIVER IN PERSON", l_shipmode = "TRUCK", l_comment = "egular courts above the" > -> 1, 
< l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 2, l_quantity = 36, l_ext

[info] welcome to sbt 1.6.0 (Oracle Corporation Java 17)
[info] loading project definition from T:\sdql\project
[info] loading settings for project sdql from build.sbt ...
[info] set current project to sdql (in build file:/T:/sdql/)
[info]
[info] Here are some highlights of this release:
[info]   - Improved JDK 17 support
[info]   - Improved Build Server Protocol (BSP) support
[info]   - Tab completion of global keys
[info] See https://eed3si9n.com/sbt-1.6.0 for full release notes.
[info] Hide the banner for this release by running `skipBanner`.
[error] server failed to start on local:sbt-server-ec9dcaf7bfc5935dead3. java.io.IOException: Could not create lock for \\.\pipe\sbt-server-ec9dcaf7bfc5935dead3_lock, error 5
>....[info] running sdql.driver.Main interpret T:/UG4-Proj/sdql_scripts/q.sdql
{<n_name = MOROCCO, revenue = 122533.0858> -> 1}
[success] Total time: 1 s, completed 2022年7月16日 上午1:10:03


{<n_name = MOROCCO, revenue = 122533.0858> -> 1}

# TPCH - 6

In [5]:
import pysdql

var1 = '1993-01-01'
var2 = '1994-01-01'  # var1 + 1 year
discount = 0.07
var3 = discount - 0.01
var4 = discount + 0.01
var5 = 25

db_driver = pysdql.db_driver(db_path=r'T:/sdql')

lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)

r = lineitem[(lineitem['l_shipdate'] >= var1) & (lineitem['l_shipdate'] < var2)
             & (lineitem['l_discount'] > var3) & (lineitem['l_discount'] < var4)
             & (lineitem['l_quantity'] < var5)]

r = r.aggr(revenue=(r['l_extendedprice'] * r['l_discount'], 'sum'))

db_driver.run(r)

let lineitem = { < l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 1, l_quantity = 17, l_extendedprice = 15317.0, l_discount = 0.04, l_tax = 0.02, l_returnflag = "N", l_linestatus = "O", l_shipdate = 19960313, l_commitdate = 19960212, l_receiptdate = 19960322, l_shipinstruct = "DELIVER IN PERSON", l_shipmode = "TRUCK", l_comment = "egular courts above the" > -> 1, 
< l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 2, l_quantity = 36, l_extendedprice = 32436.0, l_discount = 0.09, l_tax = 0.06, l_returnflag = "N", l_linestatus = "O", l_shipdate = 19960412, l_commitdate = 19960228, l_receiptdate = 19960420, l_shipinstruct = "TAKE BACK RETURN", l_shipmode = "MAIL", l_comment = "ly final dependencies: slyly bold " > -> 1, 
< l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 3, l_quantity = 8, l_extendedprice = 7208.0, l_discount = 0.1, l_tax = 0.02, l_returnflag = "R", l_linestatus = "O", l_shipdate = 19960129, l_commitdate = 19960305, l_receiptdate = 19

{<revenue = 2162.4> -> 1}

# TPCH - 10

In [6]:
db_driver = pysdql.driver(db_path=r'T:/sdql')

customer = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/customer.tbl', header=pysdql.CUSTOMER_COLS)
orders = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/orders.tbl', header=pysdql.ORDERS_COLS)
lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)
nation = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/nation.tbl', header=pysdql.NATION_COLS)

r = pysdql.merge(customer, orders, lineitem, nation,
                 on=(customer['c_custkey'] == orders['o_custkey'])
                    & (lineitem['l_orderkey'] == orders['o_custkey'])
                    & (customer['c_nationkey'] == nation['n_nationkey'])
                 )

r = r[(orders['o_orderdate'] >= 19960101)
      & (orders['o_orderdate'] < 19960401)
      & (lineitem['l_returnflag'] == 'R')]

r = r.groupby(['c_custkey', 'c_name', 'c_acctbal', 'c_phone', 'n_name', 'c_address', 'c_comment']).aggr(
    revenue=((lineitem['l_extendedprice'] * (1 - lineitem['l_discount'])), 'sum'))

db_driver.run(r)

let customer = { < c_custkey = 1, c_name = "Customer#000000001", c_address = "IVhzIApeRb ot,c,E", c_nationkey = 15, c_phone = "25-989-741-2988", c_acctbal = 711.56, c_mktsegment = "BUILDING", c_comment = "to the even, regular platelets. regular, ironic epitaphs nag e" > -> 1 }
let orders = { < o_orderkey = 1, o_custkey = 1, o_orderstatus = "O", o_totalprice = 124828.11, o_orderdate = 19960102, o_orderpriority = "5-LOW", o_clerk = "Clerk#000000951", o_shippriority = 0, o_comment = "nstructions sleep furiously among " > -> 1 }
let lineitem = { < l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 1, l_quantity = 17, l_extendedprice = 15317.0, l_discount = 0.04, l_tax = 0.02, l_returnflag = "N", l_linestatus = "O", l_shipdate = 19960313, l_commitdate = 19960212, l_receiptdate = 19960322, l_shipinstruct = "DELIVER IN PERSON", l_shipmode = "TRUCK", l_comment = "egular courts above the" > -> 1, 
< l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 2, l_quantity = 36, l_ext

[info] welcome to sbt 1.6.0 (Oracle Corporation Java 17)
[info] loading project definition from T:\sdql\project
[info] loading settings for project sdql from build.sbt ...
[info] set current project to sdql (in build file:/T:/sdql/)
[info]
[info] Here are some highlights of this release:
[info]   - Improved JDK 17 support
[info]   - Improved Build Server Protocol (BSP) support
[info]   - Tab completion of global keys
[info] See https://eed3si9n.com/sbt-1.6.0 for full release notes.
[info] Hide the banner for this release by running `skipBanner`.
[error] server failed to start on local:sbt-server-ec9dcaf7bfc5935dead3. java.io.IOException: Could not create lock for \\.\pipe\sbt-server-ec9dcaf7bfc5935dead3_lock, error 5
>....[info] running sdql.driver.Main interpret T:/UG4-Proj/sdql_scripts/q.sdql
{<c_custkey = 1, c_name = Customer#000000001, c_acctbal = 711.56, c_phone = 25-989-741-2988, n_name = MOROCCO, c_address = IVhzIApeRb ot,c,E, c_comment = to the even, regular platelets. regular,

{<c_custkey = 1, c_name = Customer#000000001, c_acctbal = 711.56, c_phone = 25-989-741-2988, n_name = MOROCCO, c_address = IVhzIApeRb ot,c,E, c_comment = to the even, regular platelets. regular, ironic epitaphs nag e, revenue = 6487.2> -> 1}

# TPCH - 11

In [7]:
db_driver = pysdql.driver(db_path=r'T:/sdql')

partsupp = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/partsupp.tbl', header=pysdql.PARTSUPP_COLS)
supplier = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/supplier.tbl', header=pysdql.SUPPLIER_COLS)
nation = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/nation.tbl', header=pysdql.NATION_COLS)

s = pysdql.merge(partsupp, supplier, nation,
                 on=(partsupp['ps_suppkey'] == supplier['s_suppkey'])
                    & (supplier['s_nationkey'] == nation['n_nationkey']),
                 name='S'
                 )

s = s[(nation['n_name'] == 'MOROCCO')]

agg_val = (s['ps_supplycost'] * s['ps_availqty'] * 1).sum()

# FROM
r = pysdql.merge(partsupp, supplier, nation,
                 on=(partsupp['ps_suppkey'] == supplier['s_suppkey'])
                    & (supplier['s_nationkey'] == nation['n_nationkey']),
                 name='R'
                 )

# WHERE
r = r[(nation['n_name'] == 'MOROCCO')]

# GOURPBY HAVING
r = r.groupby(['ps_partkey']).filter(lambda x: (x['ps_supplycost'] * x['ps_availqty']).sum() > agg_val)

# SELECT GROUPBY AGGREGATION
r = r.groupby(['ps_partkey']).aggr(value=(r['ps_supplycost'] * r['ps_availqty'], 'sum'))

db_driver.run(r)

let partsupp = { < ps_partkey = 1, ps_suppkey = 1, ps_availqty = 3325, ps_supplycost = 771.64, ps_comment = ", even theodolites. regular, final theodolites eat after the carefully pending foxes. furiously regular deposits sleep slyly. carefully bold realms above the ironic dependencies haggle careful" > -> 1, 
< ps_partkey = 1, ps_suppkey = 1, ps_availqty = 8076, ps_supplycost = 993.49, ps_comment = "ven ideas. quickly even packages print. pending multipliers must have to are fluff" > -> 1, 
< ps_partkey = 1, ps_suppkey = 1, ps_availqty = 3956, ps_supplycost = 337.09, ps_comment = "after the fluffily ironic deposits? blithely special dependencies integrate furiously even excuses. blithely silent theodolites could have to haggle pending, express requests; fu" > -> 1, 
< ps_partkey = 1, ps_suppkey = 1, ps_availqty = 4069, ps_supplycost = 357.84, ps_comment = "al, regular dependencies serve carefully after the quickly final pinto beans. furiously even deposits sleep quickly final, silent 

[info]
[info] Here are some highlights of this release:
[info]   - Improved JDK 17 support
[info]   - Improved Build Server Protocol (BSP) support
[info]   - Tab completion of global keys
[info] See https://eed3si9n.com/sbt-1.6.0 for full release notes.
[info] Hide the banner for this release by running `skipBanner`.
[error] server failed to start on local:sbt-server-ec9dcaf7bfc5935dead3. java.io.IOException: Could not create lock for \\.\pipe\sbt-server-ec9dcaf7bfc5935dead3_lock, error 5
>....[info] running sdql.driver.Main interpret T:/UG4-Proj/sdql_scripts/q.sdql
{<ps_partkey = 1, value = 1.3378707239999998E7> -> 1}
[success] Total time: 1 s, completed 2022年7月16日 上午1:10:21


{<ps_partkey = 1, value = 1.3378707239999998E7> -> 1}

# TPCH - 17

In [8]:
db_driver = pysdql.driver(db_path=r'T:/sdql')

lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)
part = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/part.tbl', header=pysdql.PART_COLS)

part_agg = lineitem.groupby(['l_partkey']) \
    .aggr(agg_partkey=lineitem['l_partkey'], avg_quantity=(0.2 * lineitem['l_quantity'], 'avg')) \
    .rename('part_agg')

r = pysdql.merge(lineitem, part, part_agg,
                 on=((part['p_partkey'] == lineitem['l_partkey'])
                     & (part_agg['agg_partkey'] == lineitem['l_partkey']))
                 )

r = r[(part['p_brand'] == 'Brand#13')
      & (part['p_container'] == 'JUMBO PKG')
      & (lineitem['l_quantity'] < part_agg['avg_quantity'])]

r = r.aggr(avg_yearly=((lineitem['l_extendedprice'] / 7.0), 'sum'))

db_driver.run(r)

let lineitem = { < l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 1, l_quantity = 17, l_extendedprice = 15317.0, l_discount = 0.04, l_tax = 0.02, l_returnflag = "N", l_linestatus = "O", l_shipdate = 19960313, l_commitdate = 19960212, l_receiptdate = 19960322, l_shipinstruct = "DELIVER IN PERSON", l_shipmode = "TRUCK", l_comment = "egular courts above the" > -> 1, 
< l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 2, l_quantity = 36, l_extendedprice = 32436.0, l_discount = 0.09, l_tax = 0.06, l_returnflag = "N", l_linestatus = "O", l_shipdate = 19960412, l_commitdate = 19960228, l_receiptdate = 19960420, l_shipinstruct = "TAKE BACK RETURN", l_shipmode = "MAIL", l_comment = "ly final dependencies: slyly bold " > -> 1, 
< l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 3, l_quantity = 8, l_extendedprice = 7208.0, l_discount = 0.1, l_tax = 0.02, l_returnflag = "R", l_linestatus = "O", l_shipdate = 19960129, l_commitdate = 19960305, l_receiptdate = 19

{<avg_yearly = 398.15142857142854> -> 1}

# TPCH - 19

In [9]:
db_driver = pysdql.driver(db_path=r'T:/sdql')

lineitem = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/lineitem.tbl', header=pysdql.LINEITEM_COLS)
part = pysdql.read_tbl(path=r'T:/UG4-Proj/datasets/part.tbl', header=pysdql.PART_COLS)

c1 = (part['p_brand'] == 'Brand#13') \
     & (part['p_container'].isin(('SM CASE', 'SM BOX', 'SM PACK', 'SM PKG'))) \
     & (lineitem['l_quantity'] >= 15) \
     & (lineitem['l_quantity'] <= 25) \
     & (part['p_size'] > 1) \
     & (part['p_size'] < 5) \
     & (lineitem['l_shipmode'].isin(('AIR', 'AIR REG'))) \
     & (lineitem['l_shipinstruct'] == 'DELIVER IN PERSON')
c2 = (part['p_brand'] == 'Brand#13') \
     & (part['p_container'].isin(('MED BAG', 'MED BOX', 'MED PKG', 'MED PACK'))) \
     & (lineitem['l_quantity'] >= 15) \
     & (lineitem['l_quantity'] <= 25) \
     & (part['p_size'] > 1) \
     & (part['p_size'] < 10) \
     & (lineitem['l_shipmode'].isin(('AIR', 'AIR REG'))) \
     & (lineitem['l_shipinstruct'] == 'DELIVER IN PERSON')
c3 = (part['p_brand'] == 'Brand#13') \
     & (part['p_container'].isin(('LG CASE', 'LG BOX', 'LG PACK', 'LG PKG'))) \
     & (lineitem['l_quantity'] >= 15) \
     & (lineitem['l_quantity'] <= 25) \
     & (part['p_size'] > 1) \
     & (part['p_size'] < 15) \
     & (lineitem['l_shipmode'].isin(('AIR', 'AIR REG'))) \
     & (lineitem['l_shipinstruct'] == 'DELIVER IN PERSON')

r = pysdql.merge(lineitem, part, on=(part['p_partkey'] == lineitem['l_partkey']))

r = r[c1 | c2 | c3]

r = r.aggr(revenue=((r['l_extendedprice'] * (1 - r['l_discount'])), 'sum'))

db_driver.run(r)

let lineitem = { < l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 1, l_quantity = 17, l_extendedprice = 15317.0, l_discount = 0.04, l_tax = 0.02, l_returnflag = "N", l_linestatus = "O", l_shipdate = 19960313, l_commitdate = 19960212, l_receiptdate = 19960322, l_shipinstruct = "DELIVER IN PERSON", l_shipmode = "TRUCK", l_comment = "egular courts above the" > -> 1, 
< l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 2, l_quantity = 36, l_extendedprice = 32436.0, l_discount = 0.09, l_tax = 0.06, l_returnflag = "N", l_linestatus = "O", l_shipdate = 19960412, l_commitdate = 19960228, l_receiptdate = 19960420, l_shipinstruct = "TAKE BACK RETURN", l_shipmode = "MAIL", l_comment = "ly final dependencies: slyly bold " > -> 1, 
< l_orderkey = 1, l_partkey = 1, l_suppkey = 1, l_linenumber = 3, l_quantity = 8, l_extendedprice = 7208.0, l_discount = 0.1, l_tax = 0.02, l_returnflag = "R", l_linestatus = "O", l_shipdate = 19960129, l_commitdate = 19960305, l_receiptdate = 19

{<revenue = 16213.470000000001> -> 1}